# Task 1 f, Calculating reserve factors against column buckling using Euler-Johnson

## We need from the data file:
- volume of the element
- Iyy calculated with `height_str, width_str, thickness_web, thickness_flange, thickness_skin, stringer_pitch`
- dimensions
- `EulerJohnson(EModulus, I_y, area, length, height_str, thickness_flange, thickness_web, radius, sigma_yield, sigma_applied, c=1)`

## Please enter your name (daniel, fabian, felix, yannis)

In [30]:
name = 'yannis'

## Imports

In [31]:
import pandas as pd
import numpy as np
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.helpers import *
from optimization.generation import *
from optimization.columnBuckReverse import *

In [32]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

## Constants

In [33]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}. Please verify.")

stringer_pitch = 200
effective_width = stringer_pitch/2

panel_element_length = 750/3
stringer_element_length = 750/3


Your personal data is: sigma_yield = 490, EModulus = 65241.07. Please verify.


# Importing necessary files 

## Import everything for panels 

In [34]:
# Import panel properties and match the elements to the respective panel
paneldf = pd.read_csv(f'../data/{name}/panel.csv')
panelPropertiesdf = pd.read_csv(f'../data/{name}/properties/panel_properties.csv', index_col=0)
panelPropertiesdf = panelPropertiesdf.drop(['mass', 'Component Name'], axis=1)
paneldf = pd.merge(paneldf, panelPropertiesdf, on='Element ID', how='left', suffixes=('_caller', '_other'))
paneldf = paneldf.drop(['sigmaXY', 'sigmaYY'], axis=1)
paneldf.head(5)

,Element ID,Component Name,sigmaXX,Load Case,thickness
0,1,panel1,-69.340,Subcase 1 (LC1),4.97
1,2,panel1,-81.340,Subcase 1 (LC1),4.97
2,3,panel1,-70.443,Subcase 1 (LC1),4.97
3,4,panel2,-96.966,Subcase 1 (LC1),5.19
4,5,panel2,-89.099,Subcase 1 (LC1),5.19


## Add a volume column to the panels 

In [35]:
paneldf['Volume'] = paneldf.apply(panel_element_volume, elementLength=panel_element_length, elementWidth=stringer_pitch, axis=1)
paneldf.head(3)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume
0,1,panel1,-69.340,Subcase 1 (LC1),4.97,124250.0
1,2,panel1,-81.340,Subcase 1 (LC1),4.97,124250.0
2,3,panel1,-70.443,Subcase 1 (LC1),4.97,124250.0


## Import everything for stringers 

In [36]:
# Open and match stringer properties 
stringerdf = pd.read_csv(f'../data/{name}/stringer.csv')
# Now add the stringer properties '../data/{name}/stringer_properties.csv
stringerPropertiesdf = pd.read_csv(f'../data/{name}/properties/stringer_properties.csv', index_col=0)
stringerPropertiesdf = stringerPropertiesdf.reset_index()
stringerPropertiesdf.rename(columns={'beamsects': 'Component Name', 'beamsect_dim1': 'dim1', 'beamsect_dim2': 'dim2', 'beamsect_dim3': 'dim3', 'beamsect_dim4': 'dim4'}, inplace=True)
# Add "stringer" prefix to Component Name
stringerPropertiesdf['Component Name'] = 'stringer' + stringerPropertiesdf['Component Name'].astype(str)
# Merge the dataframes
stringerdf = pd.merge(stringerdf, stringerPropertiesdf, on='Component Name', how='left', suffixes=('_caller', '_other'))
stringerdf

,Element ID,Component Name,sigmaXX,Load Case,dim1,dim2,dim3,dim4
0,40,stringer1,-71.096,Subcase 1 (LC1),26.90,4.91,21.34,14.95
1,41,stringer1,-68.223,Subcase 1 (LC1),26.90,4.91,21.34,14.95
2,42,stringer1,-73.429,Subcase 1 (LC1),26.90,4.91,21.34,14.95
3,43,stringer2,-75.797,Subcase 1 (LC1),26.77,3.22,19.78,14.66
4,44,stringer2,-79.653,Subcase 1 (LC1),26.77,3.22,19.78,14.66
...,...,...,...,...,...,...,...,...
76,62,stringer8,-26.462,Subcase 3 (LC3),26.77,3.22,19.78,14.66
77,63,stringer8,-18.253,Subcase 3 (LC3),26.77,3.22,19.78,14.66
78,64,stringer9,-28.657,Subcase 3 (LC3),26.90,4.91,21.34,14.95
79,65,stringer9,-22.665,Subcase 3 (LC3),26.90,4.91,21.34,14.95


## Add volume to the stringer elements

In [37]:
stringerdf['Volume']= stringerdf.apply(stringer_element_volume, elementLength = stringer_element_length, axis=1)
stringerdf.head(3)

,Element ID,Component Name,sigmaXX,Load Case,dim1,dim2,dim3,dim4,Volume
0,40,stringer1,-71.096,Subcase 1 (LC1),26.9,4.91,21.34,14.95,116882.55
1,41,stringer1,-68.223,Subcase 1 (LC1),26.9,4.91,21.34,14.95,116882.55
2,42,stringer1,-73.429,Subcase 1 (LC1),26.9,4.91,21.34,14.95,116882.55


# Now we rearrange the structure a bit

## First we split the 3 loadcases 

In [38]:
loadCase1dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 1 (LC1)']
loadCase2dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 2 (LC2)']
loadCase3dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 3 (LC3)']
loadCase1dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 1 (LC1)']
loadCase2dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 2 (LC2)']
loadCase3dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 3 (LC3)']
print(loadCase1dfPanel.head(5))
print(loadCase1dfStringer.head(5))

   Element ID Component Name  sigmaXX        Load Case  thickness    Volume
0           1         panel1  -69.340  Subcase 1 (LC1)       4.97  124250.0
1           2         panel1  -81.340  Subcase 1 (LC1)       4.97  124250.0
2           3         panel1  -70.443  Subcase 1 (LC1)       4.97  124250.0
3           4         panel2  -96.966  Subcase 1 (LC1)       5.19  129750.0
4           5         panel2  -89.099  Subcase 1 (LC1)       5.19  129750.0
   Element ID Component Name  sigmaXX        Load Case   dim1  dim2   dim3  \
0          40      stringer1  -71.096  Subcase 1 (LC1)  26.90  4.91  21.34   
1          41      stringer1  -68.223  Subcase 1 (LC1)  26.90  4.91  21.34   
2          42      stringer1  -73.429  Subcase 1 (LC1)  26.90  4.91  21.34   
3          43      stringer2  -75.797  Subcase 1 (LC1)  26.77  3.22  19.78   
4          44      stringer2  -79.653  Subcase 1 (LC1)  26.77  3.22  19.78   

    dim4     Volume  
0  14.95  116882.55  
1  14.95  116882.55  
2  14.95 

# Now we need to combine the panels and the stringers

In [39]:
# For every loadcase
# Efficiently combine panels for load case 1 
panel_groups_1 = []
for i in range(1, 10):
    df1 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase1dfStringer[loadCase1dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_1.extend([df1, df2, df3])
lc1combined = pd.concat(panel_groups_1, ignore_index=True)
lc1combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-69.340,Subcase 1 (LC1),4.97,124250.00,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-81.340,Subcase 1 (LC1),4.97,124250.00,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-70.443,Subcase 1 (LC1),4.97,124250.00,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-96.966,Subcase 1 (LC1),5.19,129750.00,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-89.099,Subcase 1 (LC1),5.19,129750.00,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-95.415,Subcase 1 (LC1),5.19,129750.00,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-71.096,Subcase 1 (LC1),NaN,116882.55,stiffener1,26.9,4.91,21.34,14.95
7,41,stringer1,-68.223,Subcase 1 (LC1),NaN,116882.55,stiffener1,26.9,4.91,21.34,14.95
8,42,stringer1,-73.429,Subcase 1 (LC1),NaN,116882.55,stiffener1,26.9,4.91,21.34,14.95
9,4,panel2,-96.966,Subcase 1 (LC1),5.19,129750.00,stiffener2,NaN,NaN,NaN,NaN


In [40]:
# Efficiently combine panels for load case 2
panel_groups_2 = []
for i in range(1, 10):
    df1 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase2dfStringer[loadCase2dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_2.extend([df1, df2, df3])
lc2combined = pd.concat(panel_groups_2, ignore_index=True)
lc2combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-71.949,Subcase 2 (LC2),4.97,124250.00,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-83.026,Subcase 2 (LC2),4.97,124250.00,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-73.051,Subcase 2 (LC2),4.97,124250.00,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-97.075,Subcase 2 (LC2),5.19,129750.00,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-89.026,Subcase 2 (LC2),5.19,129750.00,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-95.524,Subcase 2 (LC2),5.19,129750.00,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-73.576,Subcase 2 (LC2),NaN,116882.55,stiffener1,26.9,4.91,21.34,14.95
7,41,stringer1,-77.509,Subcase 2 (LC2),NaN,116882.55,stiffener1,26.9,4.91,21.34,14.95
8,42,stringer1,-75.909,Subcase 2 (LC2),NaN,116882.55,stiffener1,26.9,4.91,21.34,14.95
9,4,panel2,-97.075,Subcase 2 (LC2),5.19,129750.00,stiffener2,NaN,NaN,NaN,NaN


In [41]:
# Efficiently combine panels for load case 3 
panel_groups_3 = []
for i in range(1, 10):
    df1 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase3dfStringer[loadCase3dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_3.extend([df1, df2, df3])
lc3combined = pd.concat(panel_groups_3, ignore_index=True)
lc3combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-21.021,Subcase 3 (LC3),4.97,124250.00,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-27.022,Subcase 3 (LC3),4.97,124250.00,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-25.417,Subcase 3 (LC3),4.97,124250.00,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-35.048,Subcase 3 (LC3),5.19,129750.00,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-29.600,Subcase 3 (LC3),5.19,129750.00,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-28.864,Subcase 3 (LC3),5.19,129750.00,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-19.357,Subcase 3 (LC3),NaN,116882.55,stiffener1,26.9,4.91,21.34,14.95
7,41,stringer1,-22.665,Subcase 3 (LC3),NaN,116882.55,stiffener1,26.9,4.91,21.34,14.95
8,42,stringer1,-28.657,Subcase 3 (LC3),NaN,116882.55,stiffener1,26.9,4.91,21.34,14.95
9,4,panel2,-35.048,Subcase 3 (LC3),5.19,129750.00,stiffener2,NaN,NaN,NaN,NaN


In [42]:
# We fill the empty spaces with zero
lc1combined = lc1combined.fillna(0)
lc2combined = lc2combined.fillna(0)
lc3combined = lc3combined.fillna(0)
lc1combined

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-69.340,Subcase 1 (LC1),4.97,124250.00,stiffener1,0.0,0.00,0.00,0.00
1,2,panel1,-81.340,Subcase 1 (LC1),4.97,124250.00,stiffener1,0.0,0.00,0.00,0.00
2,3,panel1,-70.443,Subcase 1 (LC1),4.97,124250.00,stiffener1,0.0,0.00,0.00,0.00
3,4,panel2,-96.966,Subcase 1 (LC1),5.19,129750.00,stiffener1,0.0,0.00,0.00,0.00
4,5,panel2,-89.099,Subcase 1 (LC1),5.19,129750.00,stiffener1,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
76,29,panel10,-81.340,Subcase 1 (LC1),4.97,124250.00,stiffener9,0.0,0.00,0.00,0.00
77,30,panel10,-69.340,Subcase 1 (LC1),4.97,124250.00,stiffener9,0.0,0.00,0.00,0.00
78,64,stringer9,-73.429,Subcase 1 (LC1),0.00,116882.55,stiffener9,26.9,4.91,21.34,14.95
79,65,stringer9,-68.223,Subcase 1 (LC1),0.00,116882.55,stiffener9,26.9,4.91,21.34,14.95


# Now we can aggregate the loadcases according to stiffeners

## Multiply volume and stress together for averaging 

In [43]:
lc1combined['XX * Volume'] = lc1combined['sigmaXX'] * lc1combined['Volume']
lc2combined['XX * Volume'] = lc2combined['sigmaXX'] * lc2combined['Volume']
lc3combined['XX * Volume'] = lc3combined['sigmaXX'] * lc3combined['Volume']
lc1combined.head(2)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4,XX * Volume
0,1,panel1,-69.34,Subcase 1 (LC1),4.97,124250.0,stiffener1,0.0,0.0,0.0,0.0,-8615495.0
1,2,panel1,-81.34,Subcase 1 (LC1),4.97,124250.0,stiffener1,0.0,0.0,0.0,0.0,-10106495.0


## Load case 1 

In [44]:
lc1combined = lc1combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'thickness':'max',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc1combined['sigma_XX_avg'] = lc1combined['XX * Volume'] / lc1combined['Volume']
lc1combined = lc1combined.drop(['XX * Volume'], axis=1)
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg
Stiffener,,,,,,,
stiffener1,1112647.65,5.19,26.90,4.91,21.34,14.95,-79.866336
stiffener2,995073.00,5.19,26.77,3.22,19.78,14.66,-90.369318
stiffener3,1063237.95,5.45,26.21,3.83,21.46,15.80,-90.179904
stiffener4,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.762773
stiffener5,1198032.45,6.55,24.39,3.18,19.67,14.14,-90.332951
stiffener6,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.762773
stiffener7,1063237.95,5.45,26.21,3.83,21.46,15.80,-90.179904
stiffener8,995073.00,5.19,26.77,3.22,19.78,14.66,-90.369318
stiffener9,1112647.65,5.19,26.90,4.91,21.34,14.95,-79.866336


## Load case 2

In [45]:
lc2combined = lc2combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'thickness':'max',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc2combined['sigma_XX_avg'] = lc2combined['XX * Volume'] / lc2combined['Volume']
lc2combined = lc2combined.drop(['XX * Volume'], axis=1)
lc2combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg
Stiffener,,,,,,,
stiffener1,1112647.65,5.19,26.90,4.91,21.34,14.95,-82.150636
stiffener2,995073.00,5.19,26.77,3.22,19.78,14.66,-89.620843
stiffener3,1063237.95,5.45,26.21,3.83,21.46,15.80,-89.927777
stiffener4,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.811271
stiffener5,1198032.45,6.55,24.39,3.18,19.67,14.14,-90.398162
stiffener6,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.811271
stiffener7,1063237.95,5.45,26.21,3.83,21.46,15.80,-89.927777
stiffener8,995073.00,5.19,26.77,3.22,19.78,14.66,-89.620843
stiffener9,1112647.65,5.19,26.90,4.91,21.34,14.95,-82.150636


## Load case 3

In [46]:
lc3combined = lc3combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'thickness':'max',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc3combined['sigma_XX_avg'] = lc3combined['XX * Volume'] / lc3combined['Volume']
lc3combined = lc3combined.drop(['XX * Volume'], axis=1)
lc3combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg
Stiffener,,,,,,,
stiffener1,1112647.65,5.19,26.90,4.91,21.34,14.95,-26.532864
stiffener2,995073.00,5.19,26.77,3.22,19.78,14.66,-30.022087
stiffener3,1063237.95,5.45,26.21,3.83,21.46,15.80,-29.959193
stiffener4,1080251.10,6.55,25.28,2.54,19.32,14.91,-30.152826
stiffener5,1198032.45,6.55,24.39,3.18,19.67,14.14,-30.010114
stiffener6,1080251.10,6.55,25.28,2.54,19.32,14.91,-30.152826
stiffener7,1063237.95,5.45,26.21,3.83,21.46,15.80,-29.959193
stiffener8,995073.00,5.19,26.77,3.22,19.78,14.66,-30.022087
stiffener9,1112647.65,5.19,26.90,4.91,21.34,14.95,-26.532864


# Now we add Cross-Section Properties of the combined skin and hat stringer crosssection 

In [47]:
# Load case 1
lc1combined[['I_yy', 'areaTot', 'VolumeTot']] = lc1combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
# Load case 2
lc2combined[['I_yy', 'areaTot', 'VolumeTot']] = lc2combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
# Load case 3
lc3combined[['I_yy', 'areaTot', 'VolumeTot']] = lc3combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot
Stiffener,,,,,,,,,,
stiffener1,1112647.65,5.19,26.90,4.91,21.34,14.95,-79.866336,103630.402612,1505.5302,1129147.65
stiffener2,995073.00,5.19,26.77,3.22,19.78,14.66,-90.369318,77525.097317,1347.7640,1010823.00
stiffener3,1063237.95,5.45,26.21,3.83,21.46,15.80,-90.179904,87673.496287,1464.6506,1098487.95
stiffener4,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.762773,65255.502381,1550.3348,1162751.10
stiffener5,1198032.45,6.55,24.39,3.18,19.67,14.14,-90.332951,70791.831987,1597.3766,1198032.45
stiffener6,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.762773,65255.502381,1550.3348,1162751.10
stiffener7,1063237.95,5.45,26.21,3.83,21.46,15.80,-90.179904,87673.496287,1464.6506,1098487.95
stiffener8,995073.00,5.19,26.77,3.22,19.78,14.66,-90.369318,77525.097317,1347.7640,1010823.00
stiffener9,1112647.65,5.19,26.90,4.91,21.34,14.95,-79.866336,103630.402612,1505.5302,1129147.65


# Now we calculate the columnbuckling with Euler Johnson 

In [48]:
# Load case 1EModulus, DIM1, DIM2, DIM3, sigma_yield, r
lc1combined['sigma_crip'] = lc1combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
# Load case 2
lc2combined['sigma_crip'] = lc2combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
#Load case 3
lc3combined['sigma_crip'] = lc3combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip
Stiffener,,,,,,,,,,,
stiffener1,1112647.65,5.19,26.90,4.91,21.34,14.95,-79.866336,103630.402612,1505.5302,1129147.65,490
stiffener2,995073.00,5.19,26.77,3.22,19.78,14.66,-90.369318,77525.097317,1347.7640,1010823.00,490
stiffener3,1063237.95,5.45,26.21,3.83,21.46,15.80,-90.179904,87673.496287,1464.6506,1098487.95,490
stiffener4,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.762773,65255.502381,1550.3348,1162751.10,490
stiffener5,1198032.45,6.55,24.39,3.18,19.67,14.14,-90.332951,70791.831987,1597.3766,1198032.45,490
stiffener6,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.762773,65255.502381,1550.3348,1162751.10,490
stiffener7,1063237.95,5.45,26.21,3.83,21.46,15.80,-90.179904,87673.496287,1464.6506,1098487.95,490
stiffener8,995073.00,5.19,26.77,3.22,19.78,14.66,-90.369318,77525.097317,1347.7640,1010823.00,490
stiffener9,1112647.65,5.19,26.90,4.91,21.34,14.95,-79.866336,103630.402612,1505.5302,1129147.65,490


## Add lambda_crit to the loadcases

In [49]:
lc1combined['lambda_crit'] = lc1combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)
lc2combined['lambda_crit'] = lc2combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)
lc3combined['lambda_crit'] = lc3combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)

## Add lambda to the loadcases

In [50]:
lc1combined['lambda'] = lc1combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc2combined['lambda'] = lc2combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc3combined['lambda'] = lc3combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)

## Add radius of gyration to the loadcases

In [51]:
lc1combined['r_gyr'] = lc1combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc2combined['r_gyr'] = lc2combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc3combined['r_gyr'] = lc3combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)

## Now calculate the critical stress & Reserve Factor

In [52]:
lc1combined[['sigma_crit', 'Reserve Factor']] = lc1combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc2combined[['sigma_crit', 'Reserve Factor']] = lc2combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc3combined[['sigma_crit', 'Reserve Factor']] = lc3combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
Stiffener,,,,,,,,,,,,,,,,
stiffener1,1112647.65,5.19,26.90,4.91,21.34,14.95,-79.866336,103630.402612,1505.5302,1129147.65,490,51.265756,90.398746,8.296575,78.794519,0.657720
stiffener2,995073.00,5.19,26.77,3.22,19.78,14.66,-90.369318,77525.097317,1347.7640,1010823.00,490,51.265756,98.888782,7.584278,65.845600,0.485752
stiffener3,1063237.95,5.45,26.21,3.83,21.46,15.80,-90.179904,87673.496287,1464.6506,1098487.95,490,51.265756,96.938014,7.736903,68.522403,0.506561
stiffener4,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.762773,65255.502381,1550.3348,1162751.10,490,51.265756,115.602025,6.487776,48.182567,0.353908
stiffener5,1198032.45,6.55,24.39,3.18,19.67,14.14,-90.332951,70791.831987,1597.3766,1198032.45,490,51.265756,112.660934,6.657143,50.731082,0.374401
stiffener6,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.762773,65255.502381,1550.3348,1162751.10,490,51.265756,115.602025,6.487776,48.182567,0.353908
stiffener7,1063237.95,5.45,26.21,3.83,21.46,15.80,-90.179904,87673.496287,1464.6506,1098487.95,490,51.265756,96.938014,7.736903,68.522403,0.506561
stiffener8,995073.00,5.19,26.77,3.22,19.78,14.66,-90.369318,77525.097317,1347.7640,1010823.00,490,51.265756,98.888782,7.584278,65.845600,0.485752
stiffener9,1112647.65,5.19,26.90,4.91,21.34,14.95,-79.866336,103630.402612,1505.5302,1129147.65,490,51.265756,90.398746,8.296575,78.794519,0.657720


# Generate Score output 

In [53]:

lc1combined = lc1combined.reset_index()
lc2combined = lc2combined.reset_index()
lc3combined = lc3combined.reset_index()

evaluateDf = pd.concat([lc1combined, lc2combined, lc3combined])
evaluateDf = evaluateDf.reset_index()
evaluateDf = evaluateDf.drop(['index'], axis=1)
updateDF = evaluateDf.copy()
evaluateDf

,Stiffener,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
0,stiffener1,1112647.65,5.19,26.90,4.91,21.34,14.95,-79.866336,103630.402612,1505.5302,1129147.65,490,51.265756,90.398746,8.296575,78.794519,0.657720
1,stiffener2,995073.00,5.19,26.77,3.22,19.78,14.66,-90.369318,77525.097317,1347.7640,1010823.00,490,51.265756,98.888782,7.584278,65.845600,0.485752
2,stiffener3,1063237.95,5.45,26.21,3.83,21.46,15.80,-90.179904,87673.496287,1464.6506,1098487.95,490,51.265756,96.938014,7.736903,68.522403,0.506561
3,stiffener4,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.762773,65255.502381,1550.3348,1162751.10,490,51.265756,115.602025,6.487776,48.182567,0.353908
4,stiffener5,1198032.45,6.55,24.39,3.18,19.67,14.14,-90.332951,70791.831987,1597.3766,1198032.45,490,51.265756,112.660934,6.657143,50.731082,0.374401
5,stiffener6,1080251.10,6.55,25.28,2.54,19.32,14.91,-90.762773,65255.502381,1550.3348,1162751.10,490,51.265756,115.602025,6.487776,48.182567,0.353908
6,stiffener7,1063237.95,5.45,26.21,3.83,21.46,15.80,-90.179904,87673.496287,1464.6506,1098487.95,490,51.265756,96.938014,7.736903,68.522403,0.506561
7,stiffener8,995073.00,5.19,26.77,3.22,19.78,14.66,-90.369318,77525.097317,1347.7640,1010823.00,490,51.265756,98.888782,7.584278,65.845600,0.485752
8,stiffener9,1112647.65,5.19,26.90,4.91,21.34,14.95,-79.866336,103630.402612,1505.5302,1129147.65,490,51.265756,90.398746,8.296575,78.794519,0.657720
9,stiffener1,1112647.65,5.19,26.90,4.91,21.34,14.95,-82.150636,103630.402612,1505.5302,1129147.65,490,51.265756,90.398746,8.296575,78.794519,0.639431


In [54]:
evaluateDf = evaluateDf.drop(['Volume' ,'thickness' , 'sigma_XX_avg', 'I_yy',  'areaTot', 'VolumeTot', 'sigma_crip', 'lambda_crit', 'lambda', 'r_gyr', 'sigma_crit'], axis =1)
evaluateDf['score'] = evaluateDf.apply(rf_score, axis=1)
evaluateDf

,Stiffener,dim1,dim2,dim3,dim4,Reserve Factor,score
0,stiffener1,26.90,4.91,21.34,14.95,0.657720,34648.117254
1,stiffener2,26.77,3.22,19.78,14.66,0.485752,74051.501305
2,stiffener3,26.21,3.83,21.46,15.80,0.506561,68497.155153
3,stiffener4,25.28,2.54,19.32,14.91,0.353908,114274.948647
4,stiffener5,24.39,3.18,19.67,14.14,0.374401,107452.606005
5,stiffener6,25.28,2.54,19.32,14.91,0.353908,114274.948647
6,stiffener7,26.21,3.83,21.46,15.80,0.506561,68497.155153
7,stiffener8,26.77,3.22,19.78,14.66,0.485752,74051.501305
8,stiffener9,26.90,4.91,21.34,14.95,0.657720,34648.117254
9,stiffener1,26.90,4.91,21.34,14.95,0.639431,38135.997830


## Extract score and parameters

In [55]:
score = evaluateDf['score'].sum()
dimensions = extractDimensions(evaluateDf)

evaluateDf = pd.DataFrame({
    'stringer Parameters': [dimensions],
    'score': [score],
    'minRF':[evaluateDf['Reserve Factor'].min()]
})
evaluateDf

,stringer Parameters,score,minRF
0,"[[26.9, 4.91, 21.34, 14.95], [26.77, 3.22, 19....",1.411835e+06,0.353719


## Output stringer score

In [56]:
evaluateDf.to_csv(f'../data/{name}/output/stringerScore.csv', index=False)

# Reverse Engineering 

## Collect all dimension updates 

In [57]:

updateDF[['new_dim1', 'new_dim2', 'new_dim3']] = updateDF.apply(reverseAllDims, EModulus=EModulus,
                                                                            stringerPitch=stringer_pitch,
                                                                            length=stringer_element_length*3,
                                                                             RFgoal=0.9, axis=1, result_type='expand')
updateDF

ValueError: Could not find root within given tolerance. (146.02782751662096642 > 2.16840434497100886801e-19)
Try another starting point or tweak arguments.

## Cleanup dataframe

In [ ]:
updateDF = updateDF.drop(['Volume' ,'thickness' , 'sigma_XX_avg', 'I_yy',  'areaTot', 'VolumeTot', 'sigma_crip', 'lambda_crit', 'lambda', 'r_gyr', 'sigma_crit'], axis =1)

## Keep only the largest values 

In [ ]:
updateDF = updateDF.groupby('stiffener').agg({
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
    'new_dim1': 'max',
    'new_dim2': 'max',
    'new_dim3': 'max',
    'new_dim4': 'max'
})

## Output of Reverse 

In [ ]:
updateDF.to_csv(f'../data/{name}/output/newStringerDims.csv', index=False)

# Cleanup data for output 

## Drop unenessacry columns 

In [ ]:
lc1combined = lc1combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc2combined = lc2combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc3combined = lc3combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc1combined

,Stiffener,sigma_XX_avg,I_yy,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
0,stiffener1,-81.135834,43412.818062,490,51.265756,116.804133,6.421006,47.195914,0.387793
1,stiffener2,-90.375073,51679.907230,490,51.265756,108.857361,6.889750,54.338196,0.400835
2,stiffener3,-90.810330,51580.848965,490,51.265756,112.491028,6.667198,50.884446,0.373558
3,stiffener4,-90.251739,54791.834582,490,51.265756,109.179182,6.869441,54.018329,0.399020
4,stiffener5,-90.374348,38985.298605,490,51.265756,117.155895,6.401727,46.912927,0.346064
5,stiffener6,-90.251739,54791.834582,490,51.265756,109.179182,6.869441,54.018329,0.399020
6,stiffener7,-90.810330,51580.848965,490,51.265756,112.491028,6.667198,50.884446,0.373558
7,stiffener8,-90.375073,51679.907230,490,51.265756,108.857361,6.889750,54.338196,0.400835
8,stiffener9,-81.135834,43412.818062,490,51.265756,116.804133,6.421006,47.195914,0.387793


## ROUND & Add together the load cases 

In [ ]:
# Rename colums for concat
lc1combined = lc1combined.rename(columns={'sigma_XX_avg':'XX_avg_LC1', 'sigma_crit':'sigma_crit_LC1', 'Reserve Factor':'RF_LC1', 'sigma_crip':'sigma_crip_LC1'})
lc2combined = lc2combined.rename(columns={'sigma_XX_avg':'XX_avg_LC2', 'sigma_crit':'sigma_crit_LC2', 'Reserve Factor':'RF_LC2', 'sigma_crip':'sigma_crip_LC2'})
lc3combined = lc3combined.rename(columns={'sigma_XX_avg':'XX_avg_LC3', 'sigma_crit':'sigma_crit_LC3', 'Reserve Factor':'RF_LC3', 'sigma_crip':'sigma_crip_LC3'})

In [ ]:
outputdf = pd.concat([lc1combined,lc2combined,lc3combined], axis = 1)
outputdf = outputdf.round(rounding_digits)
# After concatenation, keep only the first column of cross section propertries and drop the rest
outputdf['I_y'] = outputdf.filter(like='I_yy').iloc[:, 0]  # Take the first I_yy column
outputdf['Lambda'] = outputdf.filter(like='lambda').iloc[:, 1]  # Take the first lambda column
outputdf['Lambda_crit'] = outputdf.filter(like='lambda_crit').iloc[:, 0]  # Take the first I_yy column
outputdf['R_gyr'] = outputdf.filter(like='r_gyr').iloc[:, 0]  # Take the first I_yy column
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('I_yy')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('lambda')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('r_gyr')])
outputdf.head(10)

,Stiffener,XX_avg_LC1,sigma_crip_LC1,sigma_crit_LC1,RF_LC1,Stiffener,XX_avg_LC2,sigma_crip_LC2,sigma_crit_LC2,RF_LC2,Stiffener,XX_avg_LC3,sigma_crip_LC3,sigma_crit_LC3,RF_LC3,I_y,Lambda,Lambda_crit,R_gyr
0,stiffener1,-81.14,490,47.20,0.39,stiffener1,-82.99,490,47.20,0.38,stiffener1,-26.95,490,47.20,1.17,43412.82,116.80,51.27,6.42
1,stiffener2,-90.38,490,54.34,0.40,stiffener2,-89.63,490,54.34,0.40,stiffener2,-30.02,490,54.34,1.21,51679.91,108.86,51.27,6.89
2,stiffener3,-90.81,490,50.88,0.37,stiffener3,-90.71,490,50.88,0.37,stiffener3,-30.17,490,50.88,1.12,51580.85,112.49,51.27,6.67
3,stiffener4,-90.25,490,54.02,0.40,stiffener4,-90.02,490,54.02,0.40,stiffener4,-29.98,490,54.02,1.20,54791.83,109.18,51.27,6.87
4,stiffener5,-90.37,490,46.91,0.35,stiffener5,-90.46,490,46.91,0.35,stiffener5,-30.02,490,46.91,1.04,38985.30,117.16,51.27,6.40
5,stiffener6,-90.25,490,54.02,0.40,stiffener6,-90.02,490,54.02,0.40,stiffener6,-29.98,490,54.02,1.20,54791.83,109.18,51.27,6.87
6,stiffener7,-90.81,490,50.88,0.37,stiffener7,-90.71,490,50.88,0.37,stiffener7,-30.17,490,50.88,1.12,51580.85,112.49,51.27,6.67
7,stiffener8,-90.38,490,54.34,0.40,stiffener8,-89.63,490,54.34,0.40,stiffener8,-30.02,490,54.34,1.21,51679.91,108.86,51.27,6.89
8,stiffener9,-81.14,490,47.20,0.39,stiffener9,-82.99,490,47.20,0.38,stiffener9,-26.95,490,47.20,1.17,43412.82,116.80,51.27,6.42


# Generate output file 

In [ ]:
outputdf.to_excel(f'../data/{name}/output/processed_f.xlsx')